# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
11.516033149132  |  12.527011119424301  |  12.14148096951028
11.515119524354265  |  12.526325313024174  |  12.139652679189194
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516033149132  |  12.526101118409354  |  12.140195436380846
11.516335925490615  |  12.526325313024174  |  12.140195436380846
11.516033149132  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516033149132  |  12.527011119424301  |  12.14148096951028
11.492311443507816  |  12.526325313024174  |  12.14148096951028
11.516033149132  |  12.527011119424301  |  12.14148096951028
11.515119524354265  |  12.526127712042516  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.52607774908813  |  12.140195436380846
11.515119524354265  |  12.52692490343442  |  12.14148096951028
11.51633592549061

11.612067412579773  |  12.698959273000352  |  12.06871245332651
11.612067412579773  |  12.69887850866963  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068116712392744
11.612067412579773  |  12.69758304810916  |  12.068444123684504
11.612067412579773  |  12.69887850866963  |  12.068814847692082
11.61190351140199  |  12.698959273000352  |  12.068814847692082
11.610920344105997  |  12.69887850866963  |  12.061177597093094
11.612067412579773  |  12.698959273000352  |  12.06876347918707
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.61190351140199  |  12.698959273000352  |  12.06876347918707
11.612067412579773  |  12.698959273000352  |  12.06876347918707
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698051882902186  |  12.068107075840961
11.612067412579773  |  12.69887850866963  |  12.06869972631484
11.612067412579773  |  12.69895927300

12.634792970346426  |  12.634985859695353  |  12.634985859695353
12.634316760779873  |  12.634864676960278  |  12.634454376899578
12.634985859695353  |  12.634985859695353  |  12.629835374169149
12.634985859695353  |  12.634985859695353  |  12.633692407697316
12.631551961366005  |  12.633151280888175  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634316760779873
12.633151280888175  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.633692407697316  |  12.634985859695353
12.634985859695353  |  12.634454376899578  |  12.634985859695353
12.634792970346426  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634316760779873  |  12.634985859695353
12.634985859695353  |  12.634792970346426  |  12.634985859695353
12.634985859695353  |  12.633692407697316  |  12.634864676960278
12.634985859695353  |  12.632986482882865  |  12.634985859695353
12.634985859695353  |  12.633151280888175  |  12.633692407697316
12.634985859695353  |  12

12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953537165138442  |  12.373556798280664  |  12.795287763878669
12.953280710773807  |  12.374487945108996  |  12.795287763878669
12.953376105675439  |  12.374487945108996  |  12.795287763878669
12.953537165138442  |  12.374487945108996  |  12.795287763878669
12.953386729287281  |  12.374487945108996  |  12.795287763878669
12.953386729287281  |  12.374487945108996  |  12.795287763878669
12.953386729287281  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.373974122163656  |  12.795287763878669
12.953566949356176  |  12.372989584986573  |  12.795287763878669
12.953376105675439  |  12.374487945108996  |  12.795287763878669
12.953386729287281  |  12.373556798280664  |  12.795287763878669
12.953566949356176  |  12.373556798280664  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953386729287281  |  12.374386806208609  |  12.795287763878669
12.953566949356176  |  12

12.470932117077677  |  12.631191680976166  |  12.515138170865203
12.473224752357389  |  12.631191680976166  |  12.51540733581598
12.473224752357389  |  12.629024099331755  |  12.515792844374673
12.473224752357389  |  12.630544138844325  |  12.515674610014958
12.472866047364672  |  12.631191680976166  |  12.515674610014958
12.473224752357389  |  12.631191680976166  |  12.515674610014958
12.473224752357389  |  12.630683586167232  |  12.515792844374673
12.471407392843691  |  12.631191680976166  |  12.515674610014958
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.51521996807639
12.473224752357389  |  12.630544138844325  |  12.515538155313795
12.4703588151548  |  12.631191680976166  |  12.514926414172145
12.47208800870484  |  12.631191680976166  |  12.515792844374673
12.470830126634215  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.6311

12.353048922369696  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913137931360076  |  12.1067153110891
12.353048922369696  |  12.913278798658428  |  12.1067153110891
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353048922369696  |  12.912995200006666  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.348883475310014  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.105760279838634
12.353654080299911  |  12.91252852544888  |  12.1067153110891
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.35325090024024  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.91237289726866  |  12.106891348552676
12.35256037514762  |  12.913278798

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[77, 169]","[79, 176]","[124, 183]",11.513219,12.526726,12.141090,...,"[[124, 255, 255, 255, 255, 255, 255, 255, 255,...","[11.516033149132, 11.515119524354265, 11.51633...","[12.527011119424301, 12.526325313024174, 12.52...","[12.14148096951028, 12.139652679189194, 12.141...","[2.614159345626831, 2.7527308464050293, 3.2276...","[2524.020554860433, 2530.242156982422, 2523.78...","[50.23963131692383, 50.3015124721158, 50.23725...","[14.109874735184311, 14.099182735227124, 14.11...","[0.7681165591733521, 0.7679850764383397, 0.768...","[0.9327095784641385, 0.9325989978452299, 0.932..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[72, 124, 176]","[66, 125, 184]","[70, 133, 187]",14.746535,15.805581,15.342331,...,"[[133, 255, 255, 255, 255, 255, 255, 255, 255,...","[14.733418002881471, 14.750426649535349, 14.74...","[15.807958945983751, 15.8054162543266, 15.8052...","[15.343723829257907, 15.343435881450194, 15.34...","[9.563974380493164, 9.689697504043579, 7.75117...","[2081.5076471964517, 2080.6109415690103, 2073....","[45.623542685728076, 45.613714402238834, 45.53...","[14.947023500990559, 14.948894828313936, 14.96...","[0.7899583682164045, 0.7908130445360647, 0.790...","[0.9488865137009684, 0.9489194280203938, 0.948..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[69, 107, 144, 182]","[48, 90, 136, 184]","[69, 108, 143, 189]",17.474982,18.669938,18.097619,...,"[[108, 255, 255, 255, 255, 255, 255, 255, 255,...","[17.484773902455537, 17.454841016544734, 17.47...","[18.66497418686545, 18.662076788322135, 18.677...","[18.102841072780507, 18.10497261238136, 18.106...","[7.725879669189453, 9.685771703720093, 8.46956...","[1902.615982055664, 1945.2575403849285, 1894.3...","[43.618986485883234, 44.105073862141175, 43.52...","[15.33729220349624, 15.241032534253664, 15.356...","[0.7967824014199928, 0.8020486562446543, 0.798...","[0.9562677086659512, 0.955695449797321, 0.9567..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[57, 84, 117, 150, 183]","[35, 72, 109, 147, 186]","[69, 98, 128, 157, 192]",20.003995,21.276205,20.555262,...,"[[128, 255, 255, 255, 255, 255, 255, 255, 255,...","[19.820396168954815, 20.05405784991591, 20.072...","[21.37304654496973, 21.377510615299194, 21.354...","[20.57589105163439, 20.51856909803993, 20.5362...","[9.601742029190063, 7.894060373306274, 9.28852...","[1777.1638921101887, 1836.418743133545, 1807.3...","[42.15642171852574, 42.85345660659762, 42.5130...","[15.633528800761436, 15.491086441895598, 15.56...","[0.7979737703981419, 0.8007412372115773, 0.798...","[0.9609230332455617, 0.9595297560028261, 0.960..."
4,Lena.png,2,kapur_entropy,max,"[134, 191]","[79, 150]","[96, 142]",11.611910,12.698717,12.068410,...,"[[142, 142, 142, 142, 142, 142, 142, 142, 142,...","[11.612067412579773, 11.612067412579773, 11.61...","[12.698959273000352, 12.69887850866963, 12.698...","[12.06871245332651, 12.068814847692082, 12.068...","[9.153512716293335, 7.322970628738403, 7.22240...","[1848.1335169474285, 1847.054651896159, 1846.1...","[42.9899234350031, 42.97737372032124, 42.96667...","[15.463470175883172, 15.466006150401324, 15.46...","[0.6421766376621383, 0.6424863902841306, 0.642...","[0.8947395380304336, 0.8949590792424623, 0.894..."
5,Lena.png,3,kapur_entropy,max,"[113, 158, 201]","[59, 108, 160]","[92, 132, 170]",14.465011,15.764201,15.020937,...,"[[132, 132, 170, 132, 132, 132, 132, 132, 132,...","[14.466742950751772, 14.466071452495562, 14.46...","[15.765174849104985, 15.761904508662168, 15.76...","[15.020181316808566, 15.021549700511923, 15.02...","[9.279585361480713, 9.151463985443115, 9.17873...","[1042.2262293497722, 1042.834696451823, 1038.6...","[32.28352876235453, 32.29295118832937, 32.2273...","[17.951183621594776, 17.9486488859516, 17.9663...

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/WOA/WOA_kapur_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/WOA/WOA_kapur_RGB_dict.pkl', 'wb'))